In [24]:
import pandas as pd
import math
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ativos
tickers = [ "DIRR3.SA", "JHSF3.SA",
           "MRVE3.SA", "EVEN3.SA",
           "TRIS3.SA", "EZTC3.SA",
           "TEND3.SA", "CYRE3.SA",
        #    "SYNE3.SA", 
           "HBOR3.SA" , "GFSA3.SA","TCSA3.SA" ,
           #"CURY3.SA" , "LAVV3.SA" , "PLPL3.SA" , "MTRE3.SA" , "MDNE3.SA" , "LOGG3.SA"
            ]
period = "10y"
interval = "1wk"

In [25]:
data_dict = {}  

for ticker in tickers:
    df = yf.download(ticker, period=period, interval=interval, auto_adjust=True, progress=False)
    data_dict[ticker] = df  

subplot_titles=[ticker.replace(".SA", "") for ticker in tickers]


In [26]:
 data_dict

{'DIRR3.SA':                  Open       High        Low      Close    Volume
 Date                                                            
 2015-02-09   3.001620   3.151186   2.970675   3.151186    819075
 2015-02-16   3.151186   3.342010   3.099611   3.166658   1636844
 2015-02-23   3.140871   3.213075   2.867527   2.898472   3068645
 2015-03-02   3.022250   3.048037   2.774694   2.831425   3506751
 2015-03-09   2.836582   3.156342   2.743748   3.042879  13374520
 ...               ...        ...        ...        ...       ...
 2025-01-13  24.150000  26.110001  23.879999  25.309999  10993500
 2025-01-20  25.059999  26.889999  24.660000  26.760000  10827800
 2025-01-27  26.680000  29.120001  26.590000  28.780001   9182600
 2025-02-03  28.780001  30.309999  28.459999  29.730000  10688900
 2025-02-10  29.730000  30.610001  29.600000  30.500000   4651200
 
 [523 rows x 5 columns],
 'JHSF3.SA':                 Open      High       Low     Close    Volume
 Date                        

In [27]:
# grafico
height=800
width=800

mma = 41

fig = make_subplots(
    rows=math.ceil(len(tickers)/2), cols=2, shared_xaxes=False, 
    subplot_titles=[ticker.replace(".SA", "") for ticker in tickers],  # Remove ".SA"
    vertical_spacing=0.1
)

for i, ticker in enumerate(tickers):
    df = data_dict[ticker]  # Reutiliza os dados já baixados
    avg = df.Close.rolling(window=mma, min_periods=1).mean()
    
    row = (i // 2) + 1
    col = (i % 2) + 1

    # fig.add_trace(go.Candlestick(
    #     x=df.index, open=df.Open, high=df.High, low=df.Low, close=df.Close,
    #     name=ticker, showlegend=False), row=row, col=col)

    fig.add_trace(go.Scatter(
        x=df.index, y=avg, mode='lines',
        line=dict(color='blue', width=1),
        name="Média", showlegend=False), row=row, col=col)

fig.update_layout(title_text=f"<b>Construção civil</b><br><sub>mm{mma} do fechamento semanal</sub>",
                  height= height, width=width)
fig.update_xaxes(rangeslider_visible=False,
    tickformat="%Y",  # Mostrar apenas o ano
    dtick="M36",  # Intervalo de meses entre os ticks
    ticklabelmode="period"  # Coloca o rótulo no início do período)
)
fig.show()
